# Introduction
---
본 문서는 기본적인 인공신경망을 사용해 이진분류(binary classification) 문제를 푸는 과정을 다룹니다. kaggle 데이터 중 입문자들도 부담없이 접근할 수 있는 Titanic 데이터를 사용했습니다.데이터는 [여기](https://www.kaggle.com/c/titanic)에서 내려받을 수 있습니다.

# Import libries

In [1]:
import numpy as np
import pandas as pd

# Load data

In [2]:
df = pd.read_csv('./data/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


각각의 열(colunmn)에 대한 설명은 다음과 같습니다.


| 변수        | 정의                          | 비고                                           |
|-------------|-------------------------------|------------------------------------------------|
| PassenderId | 승객식별번호                  |                                                |
| Survived    | 생존여부                      | 0 = 사망, 1 = 생존                             |
| Pclass      | 티켓 클래스                   | 1 = 1st, 2 = 2nd, 3 = 3rd                      |
| Name        | 이름                          |                                                |
| Sex         | 성별                          |                                                |
| Age         | 나이                          |                                                |
| SibSp       | 동승한 형제자매 / 배우자의 수 |                                                |
| Parch       | 동승한 부모 / 자녀의 수       |                                                |
| Ticket      | 티켓번호                      |                                                |
| Fare        | 탑승비용                      |                                                |
| Cabin       | 객실번호                      |                                                |
| Embarked    | 승선한 항구                   | C = Cherbourg, Q = Queenstown, S = Southampton |

각각의 변수가 무엇을 의미하는지 파악했으니 이제 어떤 값을 가지고 있는지 훑어보겠습니다. 기본적으로 확인해야 할 것은 전처리가 필요한 부분이 있는지, 결측값의 존재 여부 등입니다. 먼저, 데이터의 간략한 정보요약을 보겠습니다.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


전체 891개의 행(row)가 있고 수치 데이터와 그렇지 않은 데이터가 혼재되어 있는 것을 볼 수 있습니다. '891 non-null'은 891개의 관측값이 존재한다는 것을 뜻합니다. 여담이지만 null은 독일어로 0을 뜻합니다. 따라서 'non-null'은 0이 아닌 값을 뜻하므로, 관측값이 존재한다는 의미입니다. 결측값에 대한 정보만 간략히 보고 싶다면 다음과 같은 함수를 실행합니다.

In [4]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

꽤 많은 결측값이 존재하는 것을 볼 수 있습니다. 이를 어떻게 처리할지는 아래에서 다룹니다.

# Data preprocessing
---
IT 분야에 조금이라도 발을 담궈본 사람이라면 GIGO(garbage in, garbage out)이란 말을 들어봤을 것입니다. 데이터 전처리는 결함이 있는 입력값(garbage)이 들어가는 것을 방지하면서 raw data를 모델이 학습할 수 있는 형태로 바꿔주는 과정을 뜻합니다.
## Handling missing values
---
때때로 우리가 사용할 데이터에 결측값이 존재할 수 있습니다. 이를 처리하는 간단한 방법으로는 평균값, 중앙값, 최빈값으로 대체하거나 해당 행(row)를 학습 데이터에서 제외시키는 방법이 있습니다. 더 정교한 방법들과 각각의 장단점은 대한 내용은 [여기](https://analyticsindiamag.com/5-ways-handle-missing-values-machine-learning-datasets/)를 참고하시기 바랍니다.

In [5]:
df.iloc[:, 5].replace(np.NaN, df.iloc[:, 5].mean(), inplace=True)
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,29.699118,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.000000,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.000000,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.000000,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.000000,1,0,237736,30.0708,NaN,C


In [6]:
df.iloc[:, 10].fillna('Unknown', inplace=True)
df.iloc[:, 11].fillna('Unk', inplace=True)

## Feature engineering
---
피쳐 엔지니어링이란 데이터 혹은 모델에 대한 자신의 지식(domain knowledge)을 활용하여 알고리즘이 더 잘 작동하도록 하는 피쳐를 만들어내는 과정을 일컫습니다. 어떤 문제를 풀 것인지, 그리고 어떤 모델을 사용할 것인지에 따라 적절한 피쳐의 형태는 바뀔 수 있습니다. 일반적으로, 기계 학습 모델이 완전히 임의적인 데이터로부터 우리가 바라는 무언가를 학습할 수 있는 경우는 매우 드뭅니다. 따라서, 우리는 데이터를 모델이 더 수월하게 학습할 수 있도록 표현해야 할 필요가 있습니다. 여기서는 개념만 소개하고 넘어가지만, 피쳐 엔지니어링은 책 한 권 분량이 될 만큼 방대한 내용을 포함하고 있습니다. 더 깊게 알아보고자 한다면 [Feature Engineering for Machine Learning](https://www.amazon.com/Feature-Engineering-Machine-Learning-Principles/dp/1491953241/ref=sr_1_1?ie=UTF8&qid=1538758541&sr=8-1&keywords=feature+engineering)을, 왜 피쳐 엔지니어링이 중요한가에 대한 직관적인 설명을 보고 싶다면 [Deep Learning with Python](https://www.amazon.com/Deep-Learning-Python-Francois-Chollet/dp/1617294438/ref=sr_1_1_sspa?ie=UTF8&qid=1538758726&sr=8-1-spons&keywords=deep+learning+with+python&psc=1)의 4.3.2 Feature engineering을 참고하시길 바랍니다.

In [7]:
X = df.drop(df.columns[[0, 1, 3, 8]], axis=1).values # 독립변수
y = df.iloc[:, 1].values # 종속변수

이제 우리가 풀고자 하는 문제의 독립변수와 종속변수를 정의합니다. 우리가 풀고자 하는 문제는 어떤 승객이 생존할지 판단하는 것이므로, 독립변수는 승객정보이고 종속변수는 생존여부입니다. 그런데, 승객식별번호나 이름, 티켓번호 같은 변수는 생존여부에 거의 영향을 주지 않을 것이므로 이에 해당하는 열은 미리 제외시키고 학습에 사용할 데이터를 구성합니다.

## Vectorization
---
인공신경망에서 다루는 모든 데이터(입력값, 목표값)은 반드시 수치로 표현되어야 합니다. 앞서 확인했듯이 몇몇 열은 수치로 표현되어 있지만, Sex나 Embarked와 같이 수치가 아닌 열도 존재합니다. 따라서 이를 수치 데이터로 변환하는 작업을 진행합니다. 범주형 자료를 수치형 자료로 변환하는 간단한 방법 중 하나는 각각의 범주에 고유한 숫자를 부여하는 것입니다.

In [8]:
X

array([[3, 'male', 22.0, ..., 7.25, 'Unknown', 'S'],
       [1, 'female', 38.0, ..., 71.2833, 'C85', 'C'],
       [3, 'female', 26.0, ..., 7.925, 'Unknown', 'S'],
       ...,
       [3, 'female', 29.69911764705882, ..., 23.45, 'Unknown', 'S'],
       [1, 'male', 26.0, ..., 30.0, 'C148', 'C'],
       [3, 'male', 32.0, ..., 7.75, 'Unknown', 'Q']], dtype=object)

In [9]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
X[:, 1] = label_encoder.fit_transform(X[:, 1]) # Sex
X[:, 6] = label_encoder.fit_transform(X[:, 5]) # Cabin
X[:, 7] = label_encoder.fit_transform(X[:, 7]) # Embarked

In [10]:
X

array([[3, 1, 22.0, ..., 7.25, 18, 2],
       [1, 0, 38.0, ..., 71.2833, 207, 0],
       [3, 0, 26.0, ..., 7.925, 41, 2],
       ...,
       [3, 0, 29.69911764705882, ..., 23.45, 131, 2],
       [1, 1, 26.0, ..., 30.0, 153, 0],
       [3, 1, 32.0, ..., 7.75, 30, 1]], dtype=object)

In [11]:
from sklearn.preprocessing import OneHotEncoder
X[:, 1] = label_encoder.fit_transform(X[:, 1])
one_hot_encoder = OneHotEncoder(categorical_features=[1])
X = one_hot_encoder.fit_transform(X).toarray()

## Normalization

In [12]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [13]:
X

array([[-0.73769513,  0.73769513,  0.82737724, ..., -0.50244517,
        -1.24717035,  0.58111394],
       [ 1.35557354, -1.35557354, -1.56610693, ...,  0.78684529,
         1.45116508, -1.93846038],
       [ 1.35557354, -1.35557354,  0.82737724, ..., -0.48885426,
        -0.91880149,  0.58111394],
       ...,
       [ 1.35557354, -1.35557354,  0.82737724, ..., -0.17626324,
         0.36612014,  0.58111394],
       [-0.73769513,  0.73769513, -1.56610693, ..., -0.04438104,
         0.6802121 , -1.93846038],
       [-0.73769513,  0.73769513,  0.82737724, ..., -0.49237783,
        -1.07584746, -0.67867322]])

# Build the model

In [17]:
import torch
import torch.nn as nn

class ANN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ANN, self).__init__()
        self.pipe = nn.Sequential(nn.Linear(input_dim, 6), 
                                  nn.ReLU(), 
                                  nn.Linear(6, 6), 
                                  nn.ReLU(), 
                                  nn.Linear(6, output_dim), 
                                  nn.Softmax(dim=1)
                                 )
        
    def forward(self, x):
        return self.pipe(x)

In [18]:
classifier = ANN(input_dim=X.shape[1], output_dim=2)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters())

# Train the model

In [19]:
for epoch in range(10):
    for step in range(X.shape[0]):
        feature = torch.tensor([X[step]], dtype=torch.float32, requires_grad=True)
        label = torch.tensor([y[step]], dtype=torch.long)
        
        optimizer.zero_grad()
        pred = classifier(feature)
        loss = loss_func(pred, label)
        loss.backward()
        optimizer.step()